In [2]:
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from mpl_toolkits.mplot3d import Axes3D

from sklearn import decomposition

import numpy as np

## What are autoencoders

* Multilayer artificial neural networks
* Uses `representation learning`
* Tries to mimic input at the output
<img src="http://fastforwardlabs.github.io/blog-images/miriam/miriams-figure.png">

### * Map high-dimensional data another space
### * Compression
### * Learn abstract features in an unsupervised way (labeled data is expensive)
### * Denoising and hole-filling
<img src="hole-fill.png">

## Basic autoencoder example


In [3]:
(x_train, _), (x_test, y_test) = mnist.load_data()
plt.imshow(x_train[np.random.randint(0, len(x_train))], cmap='gray')
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

### Build model

In [46]:
from keras import regularizers

encoding_dim = 32
input_img = Input(shape=(28 * 28,))
encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-50))(input_img)
# encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(28 * 28, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')


In [57]:
import keras
from IPython.display import clear_output
import matplotlib.cm as cm

class Plot(keras.callbacks.Callback):
    def __init__(self, data):
        self.data = data
        
        
    def _plot_reconstruction(self, data):
        n_images = 2
        f, axs = plt.subplots(2, n_images, figsize=(13, 6), num=1)
        for i in range(n_images):
            axs[0, i].imshow(data[i].reshape(28, 28))          
            axs[1, i].imshow(self.data[i].reshape(28, 28))
        
    def _plot_pca(self, data):
        from sklearn.decomposition import PCA
        pca = PCA(2)
        projected = pca.fit_transform(data)
        plt.figure(num=2, figsize=(6, 6))
        plt.scatter(projected[:, 0], projected[:, 1],
                    c=y_test, edgecolor='none', alpha=0.5,
                    cmap=plt.cm.get_cmap('nipy_spectral', 10))
        plt.xlabel('component 1')
        plt.ylabel('component 2')
        plt.colorbar();

    def on_batch_end(self, batch, logs=None):
        if batch % 50 == 0:
            clear_output(wait=True)

            encoded_input = Input(shape=(encoding_dim, ))
            decoder_layer = autoencoder.layers[-1]
            decoder = Model(encoded_input, decoder_layer(encoded_input))

            encoder = Model(input_img, encoded)
            encoded_imgs = encoder.predict(x_test)
            decoded_imgs = decoder.predict(encoded_imgs)
            self._plot_pca(encoded_imgs)
            self._plot_reconstruction(decoded_imgs)
            plt.show()

In [58]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[Plot((x_test))])

(10000, 784)


KeyboardInterrupt: 

In [ ]:
number_of_images = 10
encoded_imgs = encoder.predict(x_test[:number_of_images])
decoded_imgs = decoder.predict(encoded_imgs)

plt.figure(figsize=(20, 4))
for i in range(number_of_images):
    # display original
    ax = plt.subplot(2, number_of_images, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, number_of_images, i + 1 + number_of_images)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


## Variational Autoencoders

- Probabalistic approach to autoencoders
- learns a Latent Variable Model Instead of learning an arbitrary representation
- Generally no parameter tuning required

<img src="https://cdn-images-1.medium.com/max/2000/1*22cSCfmktNIwH5m__u2ffA.png">

- Vanilla autoencoder is deterministic
- Variational Autoencoder is stochastic. It learns a lattent variable $z$ from inputs $x$

> Probabilistic encoder approximating the true posterior $q(z|x)$.
***
> Generative decoder samples posterior $\hat{q}(z|x)$ aproximation, which does not rely on any particular input x.

- Learning conditional distributions is facilitated by forcing a prior.
$$  z\sim N(0,I). $$

